In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [2]:
T = np.random.rand(100)
# Q = np.random.rand(10)
Q = T[:10]

In [3]:
n = T.shape[0]
m = Q.shape[0]

Ta = np.zeros(2 * n)
Ta[:n] = T
Qr = Q[::-1]
Qra = np.zeros(2 * n)
Qra[:m] = Qr
Qraf = np.fft.fft(Qra)
Taf = np.fft.fft(Ta)
QT = np.fft.ifft(Qraf * Taf)
QT = QT[m-1:n].real

In [4]:
assert np.allclose(np.convolve(Q[::-1], T, mode="valid"), QT)

In [5]:
sum_T = 0.0
sum_T_lag = 0.0
sum_T2 = 0.0
sum_T2_lag = 0.0

mu = np.zeros(n - m + 1)
sigma = np.zeros(n - m + 1)

for k in range(n):
    sum_T += T[k]
    sum_T2 += T[k] * T[k]
    
    if k - m >= 0:
        sum_T_lag += T[k - m]
        sum_T2_lag += T[k - m] * T[k - m]
    
    mu[k - m + 1] = (sum_T - sum_T_lag) / m
    sigma[k - m + 1] = np.sqrt((sum_T2 - sum_T2_lag) / m - mu[k - m + 1] * mu[k - m + 1])

In [6]:
for i in range(n - m + 1):
    assert np.isclose(np.mean(T[i:i+m]), mu[i])
    assert np.isclose(np.std(T[i:i+m]), sigma[i])

In [69]:
def calc_dist_profile(QT, m, mu, sigma, i):
    return np.sqrt(2.0 * m * (1.0 - (QT - m * mu[i] * mu) / (m * sigma[i] * sigma)))

In [139]:
n = T.shape[0]
m = Q.shape[0]
excl_zone = int(np.ceil(m / 4))

Ta = np.zeros(2 * n)
Ta[:n] = T
Qr = Q[::-1]
Qra = np.zeros(2 * n)
Qra[:m] = Qr
Qraf = np.fft.fft(Qra)
Taf = np.fft.fft(Ta)
QT = np.fft.ifft(Qraf * Taf)
QT = QT[m-1:n].real

QT_first = QT.copy()


D = calc_dist_profile(QT, m, mu, sigma, 0)

for j in range(excl_zone):
    D[j] = np.inf

P = D

for i in range(1, n - m + 1):
# for i in range(1, 2):
    for j in range(n - m, 0, -1):
        QT[j] = QT[j - 1] - T[j - 1] * T[i - 1] + T[j + m - 1] * T[i + m - 1]
    QT[0] = QT_first[i]

    # Check sliding-window dot product is correct
    assert np.allclose(np.convolve(T[i:i+m][::-1], T, mode="valid"), QT)

    D = calc_dist_profile(QT, m, mu, sigma, i)

    # Apply exclusion zone
    for j in range(i - excl_zone, i + excl_zone):
        if 0 <= j < D.shape[0]:
            D[j] = np.inf

    P = np.minimum(P, D)

    # Check distance profile (z-normalized Euclidean distance) is correct
    TrueD = np.zeros_like(D)
    for j in range(n - m + 1):
        Ti, Tj = T[i:i+m], T[j:j+m]
        Ti, Tj = (Ti - np.mean(Ti)) / np.std(Ti), (Tj - np.mean(Tj)) / np.std(Tj)
        TrueD[j] = np.sqrt(np.sum((Ti - Tj) ** 2))

        if i - excl_zone <= j < i + excl_zone:
            TrueD[j] = np.inf

    assert np.allclose(D, TrueD)

/var/folders/4c/plh3j1bd4w1gkmbfbb4qplq40000gn/T/ipykernel_42734/1318152817.py:2: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2.0 * m * (1.0 - (QT - m * mu[i] * mu) / (m * sigma[i] * sigma)))


In [138]:
TrueP = np.array(stumpy.stump(T, m=10)[:, 0]).astype(np.float64)

assert np.allclose(P, TrueP)